In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 흡연여부 판단 Model
------------------------

For daicon basic contest   
Model developed in ubuntu 22.04, python3   
colab Pro

---------------------------
- CNN
- XGBoost
- LightGBM
---------------------------
Used k-Fold 6:2:2

### Load Data



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/smoking_py/data/test.csv')

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/smoking_py/data/train.csv')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        3000 non-null   object 
 1   나이        3000 non-null   int64  
 2   키(cm)     3000 non-null   int64  
 3   몸무게(kg)   3000 non-null   int64  
 4   BMI       3000 non-null   float64
 5   시력        3000 non-null   float64
 6   충치        3000 non-null   int64  
 7   공복 혈당     3000 non-null   int64  
 8   혈압        3000 non-null   int64  
 9   중성 지방     3000 non-null   int64  
 10  혈청 크레아티닌  3000 non-null   float64
 11  콜레스테롤     3000 non-null   int64  
 12  고밀도지단백    3000 non-null   int64  
 13  저밀도지단백    3000 non-null   int64  
 14  헤모글로빈     3000 non-null   float64
 15  요 단백      3000 non-null   int64  
 16  간 효소율     3000 non-null   float64
dtypes: float64(5), int64(11), object(1)
memory usage: 398.6+ KB


In [ ]:
train

,ID,나이,키(cm),몸무게(kg),BMI,시력,충치,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,요 단백,간 효소율,label
0,TRAIN_0000,35,170,70,24.22,1.10,1,98,40,80,1.3,211,75,120,15.9,1,1.53,1
1,TRAIN_0001,40,150,55,24.44,1.00,0,173,39,104,0.6,251,46,184,11.8,1,1.45,0
2,TRAIN_0002,60,170,50,17.30,0.75,0,96,40,61,0.8,144,43,89,15.3,1,1.04,0
3,TRAIN_0003,40,150,45,20.00,0.50,0,92,40,46,0.7,178,66,110,13.4,1,1.18,0
4,TRAIN_0004,55,155,65,27.06,1.00,0,87,42,95,0.9,232,62,151,13.8,1,1.32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,TRAIN_6995,25,170,65,22.49,1.50,0,87,45,141,1.2,184,44,112,14.9,1,1.50,0
6996,TRAIN_6996,60,165,65,23.88,0.90,0,87,45,82,0.9,184,64,103,14.3,1,1.47,1
6997,TRAIN_6997,40,180,100,30.86,1.20,0,97,44,87,0.9,178,54,107,15.6,1,1.00,0
6998,TRAIN_6998,60,150,55,24.44,0.60,0,89,57,161,0.6,157,49,76,14.4,1,1.00,0


In [ ]:
train.columns

Index(['ID', '나이', '키(cm)', '몸무게(kg)', 'BMI', '시력', '충치', '공복 혈당', '혈압',
       '중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈', '요 단백',
       '간 효소율', 'label'],
      dtype='object')

In [ ]:
train.drop('ID',axis=1,inplace=True)

In [ ]:
train.head()

,나이,키(cm),몸무게(kg),BMI,시력,충치,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,요 단백,간 효소율,label
0,35,170,70,24.22,1.10,1,98,40,80,1.3,211,75,120,15.9,1,1.53,1
1,40,150,55,24.44,1.00,0,173,39,104,0.6,251,46,184,11.8,1,1.45,0
2,60,170,50,17.30,0.75,0,96,40,61,0.8,144,43,89,15.3,1,1.04,0
3,40,150,45,20.00,0.50,0,92,40,46,0.7,178,66,110,13.4,1,1.18,0
4,55,155,65,27.06,1.00,0,87,42,95,0.9,232,62,151,13.8,1,1.32,0


In [ ]:
train.shape[0]

7000

In [ ]:
test.drop('ID',axis=1,inplace=True)

In [ ]:
test

,나이,키(cm),몸무게(kg),BMI,시력,충치,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,요 단백,간 효소율
0,40,165,55,20.20,0.90,1,98,47,75,0.5,229,59,155,13.7,1,1.73
1,65,145,50,23.78,0.50,0,99,59,98,0.6,200,65,115,12.2,1,1.00
2,40,160,75,29.30,1.00,0,105,34,232,0.6,170,50,73,15.1,1,1.33
3,30,180,90,27.78,1.35,0,78,45,218,0.9,197,55,98,15.2,1,0.89
4,50,155,55,22.89,0.75,0,116,67,139,1.0,230,66,136,15.0,1,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,40,170,90,31.14,0.85,0,94,60,146,1.1,221,55,137,14.3,1,0.72
2996,25,175,80,26.12,1.00,0,95,50,100,0.8,167,47,100,13.9,1,0.83
2997,45,155,60,24.97,1.20,1,88,32,51,0.4,186,61,114,12.8,1,1.22
2998,35,175,90,29.39,1.35,0,94,42,142,0.8,163,44,91,16.3,1,0.78


In [ ]:
train.iloc[:,:-1]

,나이,키(cm),몸무게(kg),BMI,시력,충치,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,요 단백,간 효소율
0,35,170,70,24.22,1.10,1,98,40,80,1.3,211,75,120,15.9,1,1.53
1,40,150,55,24.44,1.00,0,173,39,104,0.6,251,46,184,11.8,1,1.45
2,60,170,50,17.30,0.75,0,96,40,61,0.8,144,43,89,15.3,1,1.04
3,40,150,45,20.00,0.50,0,92,40,46,0.7,178,66,110,13.4,1,1.18
4,55,155,65,27.06,1.00,0,87,42,95,0.9,232,62,151,13.8,1,1.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,25,170,65,22.49,1.50,0,87,45,141,1.2,184,44,112,14.9,1,1.50
6996,60,165,65,23.88,0.90,0,87,45,82,0.9,184,64,103,14.3,1,1.47
6997,40,180,100,30.86,1.20,0,97,44,87,0.9,178,54,107,15.6,1,1.00
6998,60,150,55,24.44,0.60,0,89,57,161,0.6,157,49,76,14.4,1,1.00


### Make Model

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,KFold
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from xgboost.sklearn import XGBClassifier
from os.path import join

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from os.path import join
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
import lightgbm as lgb


class Kfold_Pipeline:
  def __init__(self, data, k, model):
    self.data = data
    self.k = k
    self.kfold = KFold(n_splits=self.k)
    self.train_y = data.iloc[:, -1]
    self.train_x = data.iloc[:, :-1]
    self.model = model

  def min_max_prescaler(self):
    x_scaler = MinMaxScaler()
    self.train_x = x_scaler.fit_transform(self.train_x)

  def k_model_learning(self):
    idx = 1
    for train_idx, val_idx in self.kfold.split(self.train_x, self.train_y):
      x_train, x_val = pd.DataFrame(self.train_x[train_idx]), pd.DataFrame(self.train_x[val_idx])
      y_train, y_val = pd.Series(self.train_y[train_idx]), pd.Series(self.train_y[val_idx])

      model_save_path = '/content/drive/MyDrive/2023_1st_vac/smoking_py/model_test/'
      filename = join(model_save_path, 'checkpoint.ckpt')
      checkpoint = ModelCheckpoint(filename, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=0)
      earlystopping = EarlyStopping(monitor='val_loss', patience=100)
      self.history = self.model.fit(x_train, y_train)

      pred = self.model.predict(x_val)
      real = y_val
      acclist=[]
      acc=0
      for p,r in zip(pred,real):
        if int(p)==r:
          acc+=1
      accuracy=(acc/len(real))*100
      print("\n #{}'s accuracy is {}% ".format(idx,accuracy))

      idx+=1

  def model_predicted_to_df(self, test_data):
    self.min_max_prescaler()
    self.k_model_learning()
    pred = self.model.predict(test_data)
    pred_df = pd.DataFrame(data=pred,columns=['label'])
    return pred_df

In [ ]:
test.values

array([[ 40.  , 165.  ,  55.  , ...,  13.7 ,   1.  ,   1.73],
       [ 65.  , 145.  ,  50.  , ...,  12.2 ,   1.  ,   1.  ],
       [ 40.  , 160.  ,  75.  , ...,  15.1 ,   1.  ,   1.33],
       ...,
       [ 45.  , 155.  ,  60.  , ...,  12.8 ,   1.  ,   1.22],
       [ 35.  , 175.  ,  90.  , ...,  16.3 ,   1.  ,   0.78],
       [ 40.  , 150.  ,  60.  , ...,  12.4 ,   1.  ,   1.29]])

## Modeling

In [ ]:
def Test_Model(in_dim):
  model=Sequential()
  model.add(Dense(64,input_dim=in_dim,activation='relu'))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(4,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001),metrics='accuracy')
  model.summary()

  return model


In [ ]:
import xgboost as xgb
import lightgbm as lgb

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',  # 이진 분류 문제 설정
    eval_metric='logloss',  # 평가 지표 설정
    max_depth=3,  # 트리의 깊이
    learning_rate=0.1,  # 학습률
    n_estimators=100  # 트리의 개수
)

In [ ]:
pipline=Kfold_Pipeline(data=train,k=5,model=xgb_model)
res=pipline.model_predicted_to_df(test.values)


 #0's accuracy is 71.71428571428572% 

 #1's accuracy is 72.71428571428571% 

 #2's accuracy is 72.28571428571429% 

 #3's accuracy is 72.64285714285714% 

 #4's accuracy is 73.14285714285714% 


In [ ]:
res.values

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
test

,나이,키(cm),몸무게(kg),BMI,시력,충치,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,요 단백,간 효소율
0,40,165,55,20.20,0.90,1,98,47,75,0.5,229,59,155,13.7,1,1.73
1,65,145,50,23.78,0.50,0,99,59,98,0.6,200,65,115,12.2,1,1.00
2,40,160,75,29.30,1.00,0,105,34,232,0.6,170,50,73,15.1,1,1.33
3,30,180,90,27.78,1.35,0,78,45,218,0.9,197,55,98,15.2,1,0.89
4,50,155,55,22.89,0.75,0,116,67,139,1.0,230,66,136,15.0,1,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,40,170,90,31.14,0.85,0,94,60,146,1.1,221,55,137,14.3,1,0.72
2996,25,175,80,26.12,1.00,0,95,50,100,0.8,167,47,100,13.9,1,0.83
2997,45,155,60,24.97,1.20,1,88,32,51,0.4,186,61,114,12.8,1,1.22
2998,35,175,90,29.39,1.35,0,94,42,142,0.8,163,44,91,16.3,1,0.78


In [ ]:
test_=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/smoking_py/data/test.csv')

In [ ]:
idxlist=[]
for id in test_['ID']:
  idxlist.append(id)

val_list=[]
for val in res[:]:
  val_list.append(val)

In [ ]:
res_df = pd.DataFrame()

In [ ]:
val_list

[0]